In [ ]:
#use conda env sm_ont-process_reads

In [3]:
import os
from pathlib import Path
#import pandas as pd

In [4]:
proj_dir = "/master/nplatt/sm_nanopore"
ref_fas = "{}/data/genome/Smansoni_v7.fa".format(proj_dir)

Path("{}/results".format(proj_dir)).mkdir(parents=True, exist_ok=True)

samples=["smor", 
         "smle_pzq_es", 
         "smbre", 
         "smeg"]

In [5]:
os.chdir("{}/results".format(proj_dir))

In [6]:
Path("{}/results/read_correction_canu".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/read_correction_canu".format(proj_dir))

In [ ]:
for sample in samples:

    print("###################### {} ##################".format(sample))

    in_fq = "{}/results/nanofilt/{}.filtered.fastq.gz".format(proj_dir, sample)

    !conda run -n sm_ont-process_reads canu -correct -minThreads=48 -genomesize=385m -useGrid=False -d {sample} -p {sample} -nanopore {in_fq}

In [ ]:
# ~/anaconda3/envs/sm_ont-process_reads/bin/canu -correct -minThreads=48 -minThreads=96 -genomesize=385m -useGrid=False -d smbre/ -p smbre -nanopore ../nanofilt/smbre.filtered.fastq.gz >smbre.canu.log 2>&1

# ~/anaconda3/envs/sm_ont-process_reads/bin/canu -correct -minThreads=48 -minThreads=96 -genomesize=385m -useGrid=False -d smor/ -p smor -nanopore ../nanofilt/smor.filtered.fastq.gz >smor.canu.log 2>&1

# ~/anaconda3/envs/sm_ont-process_reads/bin/canu -correct -minThreads=48 -minThreads=96 -genomesize=385m -useGrid=False -d smeg/ -p smeg -nanopore ../nanofilt/smeg.filtered.fastq.gz >smeg.canu.log 2>&1

# ~/anaconda3/envs/sm_ont-process_reads/bin/canu -correct -minThreads=48 -minThreads=96 -genomesize=385m -useGrid=False -d smle_pzq_es/ -p smle_pzq_es -nanopore ../nanofilt/smle_pzq_es.filtered.fastq.gz >smle_pzq_es.canu.log 2>&1

In [ ]:
for sample in samples:

    print("###################### {} ##################".format(sample))

    in_fas = "{}/results/read_correction_canu/{}/{}.correctedReads.fasta.gz".format(proj_dir, sample, sample)

    !conda run -n sm_ont-process_reads flye --nano-cor {in_fas} -g 385m -t 128 -i 3 -o {sample}

done


In [ ]:
mkdir /master/nplatt/sm_nanopore/results/polish_r1
cd /master/nplatt/sm_nanopore/results/polish_r1

for SAMPLE in smle_pzq_es smbre smeg smor; do
    minimap2 \
        -a \
        -map-ont \
        -t 192 \
        /master/nplatt/sm_nanopore/results/assembly/${SAMPLE}/assembly.fasta \
        /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
        >${SAMPLE}_r1.sam

    racon \
        -t 192 \
        /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
        ${SAMPLE}_r1.sam \
        /master/nplatt/sm_nanopore/results/assembly/${SAMPLE}/assembly.fasta \
        >${SAMPLE}_r1_racon.fasta
done

In [ ]:
mkdir /master/nplatt/sm_nanopore/results/polish_r2
cd /master/nplatt/sm_nanopore/results/polish_r2

for SAMPLE in smle_pzq_es smbre smeg smor; do
    minimap2 \
       -a \
       -map-ont \
       -t 48 \
       /master/nplatt/sm_nanopore/results/polish_r1/${SAMPLE}_r1_racon.fas \
       /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
       >${SAMPLE}_r2.sam 2>${SAMPLE}_r2_minimap.log &

    racon \
        -t 48 \
        /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
        ${SAMPLE}_r2.sam \
        /master/nplatt/sm_nanopore/results/polish_r1/${SAMPLE}_r1_racon.fasta \
        >${SAMPLE}_r2_racon.fas 2>${SAMPLE}_r2_racon.log &
done

In [ ]:
mkdir /master/nplatt/sm_nanopore/results/polish_r3
cd /master/nplatt/sm_nanopore/results/polish_r3

for SAMPLE in smle_pzq_es smbre smeg smor; do
    minimap2 \
        -a \
        -map-ont \
        -t 64 \
        /master/nplatt/sm_nanopore/results/polish_r2/${SAMPLE}_r2_racon.fasta \
        /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
        >${SAMPLE}_r3.sam 2>${SAMPLE}_r3_minimap.log &

done

wait 

for SAMPLE in smle_pzq_es smbre smeg smor; do
    racon \
        -t 192 \
        /master/nplatt/sm_nanopore/results/read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz \
        ${SAMPLE}_r3.sam \
        /master/nplatt/sm_nanopore/results/polish_r2/${SAMPLE}_r2_racon.fasta \
        >${SAMPLE}_r3_racon.fasta 2>${SAMPLE}_r3_racon.log &
done


In [ ]:
mkdir /master/nplatt/sm_nanopore/results/busco
cd /master/nplatt/sm_nanopore/results/busco

conda activate sm_ont-busco

for SAMPLE in smle_pzq_es smbre smeg smor; do
    busco \
        -i /master/nplatt/sm_nanopore/results/polish_r3/${SAMPLE}_r3_racon.fasta \
        --auto-lineage-euk \
        -c 48 \
        -o ${SAMPLE} \
        -m genome \
        >${SAMPLE}.busco.log
done

busco \
    -i /master/nplatt/sm_nanopore/data/genome/SM_V9.fa \
    --auto-lineage-euk \
    -c 48 \
    -o SM_V9 \
    -m genome \
    >SM_V9.busco.log


In [ ]:
cd sm_nanopore/results/synteny/
conda activate sm_ont-sibeliaz
export SATSUMA2_PATH=/master/nplatt/anaconda3/envs/sm_ont-sibeliaz/bin

#need to run repeat modeler and then repeatmasker to HARD MASK the repeats.
SatsumaSynteny2 -q ../polish_r3/smle_pzq_es_r3_racon.fasta  -t ../../data/genome/SM_V9.fa  -o smle_pzq_es -slaves 12 -threads 12 -sl_mem 64
SatsumaSynteny2 -q ../polish_r3/smor_r3_racon.fasta  -t ../../data/genome/SM_V9.fa  -o smor -slaves 12 -threads 12 -sl_mem 64
SatsumaSynteny2 -q ../polish_r3/smle_r3_racon.fasta  -t ../../data/genome/SM_V9.fa  -o smle -slaves 12 -threads 12 -sl_mem 64
SatsumaSynteny2 -q ../polish_r3/smbre_r3_racon.fasta  -t ../../data/genome/SM_V9.fa  -o smbre -slaves 12 -threads 12 -sl_mem 64

In [2]:
cmd="samtools flagstat /master/nplatt/sch_hae_scan/results/mapped_reads/sh_tzpem_SRR8284792_pe.sam"
!conda run -n samtools {cmd}

2895930 + 0 in total (QC-passed reads + QC-failed reads)
2895930 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
2340800 + 0 mapped (80.83% : N/A)
2340800 + 0 primary mapped (80.83% : N/A)
2895930 + 0 paired in sequencing
1447965 + 0 read1
1447965 + 0 read2
2285142 + 0 properly paired (78.91% : N/A)
2300692 + 0 with itself and mate mapped
40108 + 0 singletons (1.38% : N/A)
2564 + 0 with mate mapped to a different chr
2564 + 0 with mate mapped to a different chr (mapQ>=5)



In [ ]:

 ~/anaconda3/envs/sm_ont-process_reads/bin/flye --nano-cor ../read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz -g 385m -t 96  -i 3 -o ${SAMPLE} >${SAMPLE}.canu.log 2>&1 &

conda activate repeatmasker
RepeatMasker \
    -pa 12 \
    -lib ../repeatmodeler/Smansoni_v7-families.fa \
    -gff \
    -s \
    -dir $(pwd) \
    -small \
    -x \
    $REF



SmOR_Female_6_02_22_vSM_V9_sorted.vcf
schisto_pairs_04132022_vSM_V9_sorted.vcf
SmEG_Female_5_25_22_vSM_V9_sorted.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500) && AF>0.75' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pzq-es_filtered.vcf

bedtools intersect \
    -c \
    -b smle-pzq-es_filtered.vcf \
    -a ../lift/v9.bed \
    >smle-pzq-es_intersect.bed


bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-200 || SVLEN>200)' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout

#############################################################################################

#genrate reads
art_illumina \
    --len 150 \
    --fcov 100 \
    --errfree \
    -na \
    --in SM_V9.fa \
    --out sm_v9_illumina \
    >sm_v9_illumina.log 2>&1 &

nanosim-h \
    --perfect \
    -o sm_v9_ont \
    -n 5000000 \
    SM_V9.fa \
    >sm_v9_ont.log 2>&1 &

#convert to fasta
sed -n '1~4s/^@/>/p;2~4p' sm_v9_illumina.fq > sm_v9_illumina.fa

#make sure amount of data is similar
echo sm_v9_ont.fa $(cat sm_v9_ont.fa | paste  - - | cut -f2 | wc -c) >>counts &
echo sm_v9_illumina.fa $(cat sm_v9_illumina.fa | paste  - - | cut -f2 | wc -c) >>counts &




#map reads (perfectly and unambiguosly) back to the reference
for SAMPLE in sm_v9_illumina sm_v9_ont; do
    bbmap.sh \
        ref=SM_V9.fa \
        perfectmode=t \
        threads=48 \
        ambiguous=toss \
        in=${SAMPLE}.fa \
        overwrite=t \
        out=${SAMPLE}.sam \
        mappedonly=t \
        >${SAMPLE}.bbmap.log 2>&1 &
done


#calculate coverage
for SAMPLE in sm_v9_illumina sm_v9_ont; do

    echo $SAMPLE
    echo "...sort"
    samtools view -b -F 4 ${SAMPLE}.sam | samtools sort >${SAMPLE}.bam
    echo "...idx"
    samtools index ${SAMPLE}.bam
    echo "...cov"
    mosdepth \
        --threads 4 \
        ${SAMPLE} \
        ${SAMPLE}.bam
done

#how much is inaccessible per read type
for SAMPLE in sm_v9_illumina sm_v9_ont; do

    MB=$(zcat ${SAMPLE}.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}')
    echo -e $SAMPLE"\t"$MB

done

art_illumina \
--paired \
--len 150 \
--fcov 10 \
--mflen 300 \
--sdev 80 \
-na \
--in SM_V9.fa \
--out sm_v9_wes


bbmap.sh \
    ref=SM_V9.fa \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_wes1.fq \
    in2=sm_v9_wes2.fq \
    samestrandpairs=t \
    killbadpairs=t \
    overwrite=t \
    out=sm_v9_wes.sam \
    mappedonly=t \
    ignorebadquality=t

samtools view -b -F 4 sm_v9_wes.sam | samtools sort >sm_v9_wes.bam
samtools index sm_v9_wes.bam

mosdepth \
    --threads 4 \
    sm_v9_wes \
    sm_v9_wes.bam



zcat sm_v9_wes.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_wes.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'








sed -i 's/SM_V9_1/HE601624.3/' sm_v9.gff3
sed -i 's/SM_V9_2/HE601625.3/' sm_v9.gff3
sed -i 's/SM_V9_3/HE601626.3/' sm_v9.gff3
sed -i 's/SM_V9_4/HE601627.3/' sm_v9.gff3
sed -i 's/SM_V9_5/HE601628.3/' sm_v9.gff3
sed -i 's/SM_V9_6/HE601629.3/' sm_v9.gff3



bedtools intersect \
    -wb \
    -a smle-pzq-es_filtered.vcf \
    -b sm_v9.gff3





bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500) && AF>0.75' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pzq-es_filtered.vcf

bedtools intersect \
    -c \
    -b smle-pzq-es_filtered.vcf \
    -a ../lift/v9.bed








cp ../cutesv/sm_v9.gff3 .
awk '{if ($3=="exon") print $0}' sm_v9.gff3 >sm_v9_exons.gff3
cut -f1 -d";" sm_v9_exons.gff3 | cut -f1,4,5,9 | sed 's/ID=exon://' >sm_v9_exons.bed
bedtools intersect -a sm_v9_exons.bed  -b qtls_v9.bed >qtl_exons.bed


bedtools intersect -wb -a qtls_v9.bed -b smeg_filtered.vcf  >smeg_qtl_snvs.vcf
bedtools intersect -wa -b smeg_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smeg_snvs_in_qtl_exons.bed
bedtools intersect -c -b smeg_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smeg_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c

bedtools intersect -wb -a qtls_v9.bed -b smor_filtered.vcf  >smor_qtl_snvs.vcf
bedtools intersect -wa -b smor_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smor_snvs_in_qtl_exons.bed
bedtools intersect -c -b smor_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smor_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c


bedtools intersect -wb -a qtls_v9.bed -b smle-pzq-es_filtered.vcf  >smle-pzq-es_qtl_snvs.vcf
bedtools intersect -wa -b smle-pzq-es_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smle-pzq-es_snvs_in_qtl_exons.bed
bedtools intersect -c -b smle-pzq-es_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smle-pzq-es_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c









nanosim-h --perfect -o sm_v9_ont_reads -n 500000 SM_V9.fa


    minimap2 \
        -a \
        -o sm_v9_ont.sam \
        -t 48 \
        -x map-ont \
        SM_V9.fa \
        sm_v9_ont_reads.fa



samtools view -b -F 4 sm_v9_ont.sam | samtools sort >sm_v9_ont.bam

samtools index sm_v9_ont.bam

mosdepth \
    --threads 4 \
    sm_v9_ont \
    sm_v9_ont.bam

zcat sm_v9_ont.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_ont.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'


bbmap.sh \
    ref=SM_V9.fa \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_ont_reads.fa \
    overwrite=t \
    out=sm_v9_ont_bbmap.sam \
    mappedonly=t \
    ignorebadquality=t

bbmap.sh \
    ref=SM_V9.fa \
    perfectmode=t \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_ont_reads.fa \
    overwrite=t \
    out=sm_v9_ont_bbmap.sam \
    mappedonly=t


samtools view -b -F 4 sm_v9_ont_bbmap.sam | samtools sort >sm_v9_ont_bbmap.bam
samtools index sm_v9_ont_bbmap.bam

mosdepth \
    --threads 4 \
    sm_v9_ont_bbmap \
    sm_v9_ont_bbmap.bam

zcat sm_v9_ont_bbmap.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_ont_bbmap.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'

RepeatMasker -qq -pa 48 -nolow -norna -dir . -gff -lib sm_v7_tes.fas SM_V9.fa

SmEG_Female_5_25_22
schisto_pairs_04132022  ../cutesv/SmOR_Female_6_02_22_vSM_V9_sorted.vcf

for VCF in $(ls ../cutesv/*_vSM_V9_sorted.vcf); do 

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pze-es_bigsvs.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/SmEG_Female_5_25_22_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smeg_bigsvs.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/SmOR_Female_6_02_22_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smor_bigsvs.vcf

   396,000,000
39,777,694,771


In [9]:
print("here")

here
